In [1]:
from utils import dataset_manager, model_manager
import os
from flask import Flask, request, jsonify
from routes.interact_with_csv_files import csv_routes
from deep_translator import GoogleTranslator
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# data_dir = os.path.join(os.getcwd(), '..', 'data') # link to 'data' folder, remember to organize as described in Github
data_dir = '/Users/VoThinhPhat/Desktop/data'
dataset_manager = dataset_manager.Dataset(data_dir=data_dir)

Finish L13_V030 with frame_id 29217...

In [3]:
# clip vit large patch14 model
model_clip14 = model_manager.CLIP_14_model()

# remember to download model_config_file and model_file (contact vtphatt2 for link)
model_config_file = os.path.join(os.getcwd(), 'task-former', 'code', 'training', 
                                 'model_configs', 'ViT-B-16.json')
model_file = os.path.join(os.getcwd(), 'task-former', 'model', 'tsbir_model_final.pt')
model_task_former = model_manager.TASK_former_model(model_config_file=model_config_file,
                                                    model_file=model_file)

# traslate from Vietnames to English
translator = GoogleTranslator(source='vi', target='en')

Model openai-clip-vit-large-patch14 is using mps


/Users/VoThinhPhat/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Model TASK-former is using mps


/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/src/utils/model_manager.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_file, map_locatio

In [7]:
def searchByText(text_query, k = 200, discarded_videos = ""):
    submission_list = []
    text_embedding = [model_clip14.inference(text_query)]

    discarded_set = set(video.strip() for video in discarded_videos.split(','))

    dataset = dataset_manager.get_dataset()
    results = []
    for video_name, embeddings_array in dataset_manager.get_video_clip14_embedding_dict().items():
        if (video_name in discarded_set):
            continue
        
        sim_scores = cosine_similarity(text_embedding, embeddings_array).flatten()
        for index, score in enumerate(sim_scores):
            results.append((video_name, index, score))

    results.sort(key=lambda item: item[2], reverse=True)

    visited = [False] * k
    for i in range(0, k):
        if (not visited[i]):
            left = i
            right = i 
            visited[i] = True
            video_name = results[i][0]
            x = [video_name, [(dataset[video_name][results[i][1]]['filepath'], dataset[video_name][results[i][1]]['frame_id'])]]

            for j in range(i + 1, k):
                if (not visited[j] and video_name == results[j][0] and abs(results[i][1] - results[j][1]) < 12):
                    x[1].append((dataset[video_name][results[j][1]]['filepath'], dataset[video_name][results[j][1]]['frame_id']))
                    left = min(left, j)
                    right = max(right, j)
                    visited[j] = True
            
            if (len(x[1]) < 5) :
                low = max(0, left - 2)
                high = min(right + 3, len(dataset[video_name]))
                for i in range(low, left):
                    x[1].append((dataset[video_name][results[i][1]]['filepath'], dataset[video_name][results[i][1]]['frame_id']))
                for i in range(right + 1, high):
                    x[1].append((dataset[video_name][results[i][1]]['filepath'], dataset[video_name][results[i][1]]['frame_id']))

            x[1] = sorted(x[1], key=lambda a:a[1])
            submission_list.append(x)
    
    return submission_list

In [5]:
def temporalSearch(text_first_this, text_then_that, k = 100, range_size = 8, discarded_videos = ""):
    submission_list = []
    x = model_clip14.inference(text_first_this)
    y = model_clip14.inference(text_then_that)

    discarded_set = set(video.strip() for video in discarded_videos.split(','))

    results = []
    for video_name, embeddings_array in dataset_manager.get_video_clip14_embedding_dict().items():
        if (video_name in discarded_set):
            continue

        num_vectors = len(embeddings_array)
        for i in range(0, num_vectors - range_size + 1, int(range_size / 2)):
            block = embeddings_array[i:i+range_size]
            x_cos_sim = cosine_similarity([x], block[:int(0.7 * range_size)])[0]
            y_cos_sim = cosine_similarity([y], block[int(0.3 * range_size):])[0]
            block_similarity = (np.max(x_cos_sim) + np.max(y_cos_sim)) / 2
            
            results.append((block_similarity, video_name, i))

    results.sort(key=lambda x: x[0], reverse=True)
    top_results = results[:k]

    dataset = dataset_manager.get_dataset()
    for similarity, video_name, best_index in top_results:
        x = [video_name, []]
        for j in range(best_index, best_index + range_size):
            x[1].append((dataset[video_name][j]['filepath'], dataset[video_name][j]['frame_id']))
        submission_list.append(x)
    
    return submission_list

In [8]:
app = Flask(__name__)

# Register the blueprint with the main app
app.register_blueprint(csv_routes)

@app.route('/search_by_text', methods=['POST'])
def search_by_text():
    data = request.json
    search_text = data.get('searchText') 
    discarded_videos = data.get('discardedVideos')
    translated_text = translator.translate(search_text)
    submission_list = searchByText(translated_text, k=100, discarded_videos=discarded_videos)  # Ensure this returns an ordered dict if necessary

    response = jsonify({
        "translated_text": translated_text,
        "submission_list": submission_list  # Use a list instead of a dict
    })
    response.headers['Cache-Control'] = 'no-store, no-cache, must-revalidate, max-age=0'
    response.headers['Pragma'] = 'no-cache'
    response.headers['Expires'] = '0'

    return response, 200

@app.route('/temporal_search', methods=['POST'])
def temporal_search():
    # Extract the request data
    data = request.json
    text_first_this = data.get('textFirstThis')
    text_then_that = data.get('textThenThat')
    discarded_videos = data.get('discardedVideos')
    translated_first_this = translator.translate(text_first_this)
    translated_then_that = translator.translate(text_then_that)

    submission_list = temporalSearch(translated_first_this, translated_then_that, k = 30, range_size=30,
                                     discarded_videos=discarded_videos)

    # Prepare and return the response
    response = jsonify({
        "translated_first_this": translated_first_this,
        "translated_then_that": translated_then_that,
        "submission_list": submission_list
    })
    response.headers['Cache-Control'] = 'no-store, no-cache, must-revalidate, max-age=0'
    response.headers['Pragma'] = 'no-cache'
    response.headers['Expires'] = '0'

    return response, 200

app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Sep/2024 18:44:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2024 18:44:35] "GET /static/reload.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 18:44:35] "GET /static/temporal_search.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 18:44:35] "GET /static/interact_with_csv_files.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 18:44:35] "GET /static/canvasDrawing.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 18:44:35] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2024 18:44:35] "GET /static/search_mode.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 18:44:35] "GET /static/zoom.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 18:44:35] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L01/L01_V006/20073.jpg HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/2024 18:44:35] "GET /image/Users/VoThinhPhat/Desktop/data/batch1/keyframes/keyframes_L01/L01_V006/20035.jpg HTTP/1.1" 304 -
127.0.0.1 - - [27/Sep/